In [2]:
import asyncio
from pyppeteer import launch
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import nest_asyncio
import re

nest_asyncio.apply()

In [3]:
async def main(url):
    browser = await launch()
    page = await browser.newPage()
    await page.goto(url)
    
    # Get html
    
    html = await page.content()
    await browser.close()
    return html

In [4]:
links_races = []
for i in range(1,6):
    print(f'Getting page: https://bikepacking.com/bikes/bikepacking-race-rigs/page/{i}/')
    
    url = f'https://bikepacking.com/bikes/bikepacking-race-rigs/page/{i}/'
    html_response = asyncio.get_event_loop().run_until_complete(main(url))
            


        ## Load html response into BeautifulSoup

    soup = BeautifulSoup(html_response, 'html.parser')
    ul_element = soup.find('ul', class_= 'postlist')

    link_list = []
        # Extract href from each <li>
    if ul_element:
        for li_element in ul_element.find_all('li'):
                # Find the <a> tag within the <li>
                a_tag = li_element.find('a')
                
                # Extract the href attribute
                if a_tag and 'href' in a_tag.attrs:
                    link_list.append(a_tag['href'])
    
    links_races.append(link_list)
    print(f'Finished page: {i}!!')


Getting page: https://bikepacking.com/bikes/bikepacking-race-rigs/page/1/
Finished page: 1!!
Getting page: https://bikepacking.com/bikes/bikepacking-race-rigs/page/2/
Finished page: 2!!
Getting page: https://bikepacking.com/bikes/bikepacking-race-rigs/page/3/
Finished page: 3!!
Getting page: https://bikepacking.com/bikes/bikepacking-race-rigs/page/4/
Finished page: 4!!
Getting page: https://bikepacking.com/bikes/bikepacking-race-rigs/page/5/
Finished page: 5!!


In [5]:
# Flatten link_races 

flat_list = [item for sublist in links_races for item in sublist]

print(flat_list)



['https://bikepacking.com/bikes/2023-arizona-trail-race-rigs/', 'https://bikepacking.com/bikes/2023-arkansas-high-country-race-rigs/', 'https://bikepacking.com/bikes/rigs-of-the-2023-badlands-part-one/', 'https://bikepacking.com/bikes/2023-colorado-trail-race-rigs/', 'https://bikepacking.com/bikes/2023-silk-road-mountain-race-rigs/', 'https://bikepacking.com/bikes/2023-ascend-armenia-rigs/', 'https://bikepacking.com/bikes/2023-rigs-of-the-tour-divide-breakdown/', 'https://bikepacking.com/bikes/rigs-of-the-2023-tour-divide-part-2/', 'https://bikepacking.com/bikes/2023-tour-divide-rigs-part-1/', 'https://bikepacking.com/bikes/2023-trans-balkan-race-rigs/', 'https://bikepacking.com/bikes/2023-highland-trail-550-rigs/', 'https://bikepacking.com/bikes/2023-pinyons-and-pines-rigs/', 'https://bikepacking.com/bikes/2023-hellenic-mountain-race-rigs/', 'https://bikepacking.com/bikes/2023-stagecoach-400-rigs/', 'https://bikepacking.com/bikes/2022-klunk-and-float-rigs/', 'https://bikepacking.com/b

### Scrap the pages


In [6]:
print(f'Getting info from:', flat_list[0])

url = flat_list[0]
html_response = asyncio.get_event_loop().run_until_complete(main(url)) 



Getting info from: https://bikepacking.com/bikes/2023-arizona-trail-race-rigs/


In [7]:
# 
soup = BeautifulSoup(html_response, 'html.parser')
# get the h2

post_author = soup.find('div', class_='post-author')


h2_element = soup.find_all('h2')

# Names riders: extract the text from the h2
print(len(h2_element))
names_riders = []
for i in range(len(h2_element)):
    names_riders.append(h2_element[i].text)
    
print(names_riders)



42
['Miron Golfman', 'Matt Annabel', 'Ryan Bartelme', 'Thomas Duehrsen', 'Elliot DuMont', 'Johnny Fitzgerald', 'Zack Freundlich', 'Matan Guttmann', 'Jason Hanson', 'Jennifer Hanson', 'Danny Harper', 'Alexandera Houchin', 'Aaron Johnson', 'Matt Lind', 'Eric Melby', 'John O’Malley', 'Eric Payne', 'Henley Phillips', 'Karin Pocock', 'Steven Powers', 'Johnny Price', 'Ian Quigley', 'Katya Rakhmatulina', 'Dave Roch (Donut King)', 'Kat Roch', 'Elizabeth Sampey', 'Alex Schultz', 'Indiana Schulz', 'Pete Schuster', 'Bill Shand', 'Ryan Sigsbey', 'Steven Spicka', 'Josh Stair', 'Katie Strempke', 'Jeff Taylor', 'Trevor Thornton', 'Kristen Tonsager', 'Josh Tucker', 'Dylan Turner', 'Ethan Washam', 'David Wilson', 'Marissa Wright']


In [16]:
h4_element = soup.find_all('h4', style="margin-top:0px;")
print(len(h4_element))

demographics  = []
for i in range(len(h4_element)):
    demographics.append(h4_element[i].text)



# Split each element in demographics into two lists age and country

age = []
location = []

for item in demographics:
    # Extract age using regular expression
    age_match = re.search(r'Age (\d+)', item)
    if age_match:
        age.append(int(age_match.group(1)))
    else:
        age.append(None)

    # Extract location using regular expression
    location_match = re.search(r'/ (.+)', item)
    if location_match:
        location.append(location_match.group(1))
    else:
        location.append(None)


42


In [21]:
location

['Anchorage Alaska (USA)',
 'Carbondale, Colorado (USA)',
 'Tucson, Arizona (USA)',
 'Thun (Switzerland)',
 'Tucson, Arizona (USA)',
 'Helena, Montana (USA)',
 'McCall, Idaho (USA)',
 'North Vancouver, British Columbia (Canada)',
 'Florence, Arizona (USA)',
 'Florence, Arizona (USA)',
 'Phoenix, Arizona (USA)',
 'Cloquet, Minnesota (USA)',
 'Truckee, California (USA)',
 'Albuquerque, New Mexico (USA)',
 'Richland/Sequim, Washington (USA)',
 'Scranton, Pennsylvania (USA)',
 'Silver City, New Mexico (USA)',
 'Tucson, Arizona (USA)',
 'Blue River, Colorado (USA)',
 'Goodyear, Arizona (USA)',
 'The Great Lakes (USA)',
 'Edmonton, Alberta (Canada)',
 'Oakland, California (USA)',
 'Eagle River, Alaska (USA)',
 'Eagle River, Alaska (USA)',
 'Tucson, Arizona (USA)',
 'Marquette, Michigan (USA)',
 'Salida, Colorado (USA)',
 'Littleton, Colorado (USA)',
 'Schumacher, Ontario (Canada)',
 'Brevard, North Carolina (USA)',
 'Phoenix, Arizona (USA)',
 'Philadelphia, Pennsylvania (USA)',
 'Goldie the 

In [22]:
country_pattern = re.compile(r'\((.*?)\)')

# Extract country information for each location in the list
country_list = [country_pattern.search(item).group(1) for item in location if country_pattern.search(item)]
country_list


['USA',
 'USA',
 'USA',
 'Switzerland',
 'USA',
 'USA',
 'USA',
 'Canada',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'Canada',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'Canada',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA']

In [9]:
# Get the bike name

p_element = soup.find_all('p')

keywords = ['BIKE', 'BAGS', 'GEAR HIGHLIGHTS']

kit_list = []

for i, p_element in enumerate(p_element):
    paragraph_text = p_element.get_text()

    # Check if any of the keywords is present in the paragraph
    if any(keyword in paragraph_text for keyword in keywords):
        print(f"{i}: {paragraph_text}")
        print('----------------------')
        kit_list.append(paragraph_text)




15: BIKE: I’m riding a Santa Cruz Blur, with SRAM XO1 drive train, code RSC brakes, DT Swiss 232 ONE 120mm fork, RockShox Sid Ultimate deluxe 150mm shock, Mason X HUNT search 29 MTB Dynamo disc wheelset, Schwalbe wicked will tires.
BAGS: DTF dreams custom handlebar bag, Woolf Tooth b-rad bag, Bedrock bags dropper saddle bag, Osprey Syncro 20L backpack.
GEAR HIGHLIGHTS: KLite Ultra Racing Lighting System w/ Bar switch and USB Charge system all powered by my HUNT Dynamo wheels. I am racing to support the Vermont Bicycle Shop flood relief fundraiser, make a donation here. 
----------------------
16: BIKE: I’ll be piloting “Badger,” my singlespeed Why Cycles Wayward all the way to Stateline. Badger is geared 32x22T with an Absolute oval up front and a Problem Solvers SRAM adapter in back. The front wheel is a Mason X Hunt with a SON dynamo hub. The rear wheel is an Industry Nine Hydra. Both sport thick Maxxis Rekon cactus-flingers of the 29×2.6″ variety.
BAGS: A much loved Rogue Panda Cane

In [10]:


bike_section, bags_section, gear_section = map(str.strip, kit_list[0].split('\n'))

# Extract information after each header
bike_info = bike_section.split('BIKE: ')[1]
bags_info = bags_section.split('BAGS: ')[1]
gear_info = gear_section.split('GEAR HIGHLIGHTS: ')[1]


In [11]:


# Define the headers
headers = ['BIKE:', 'BAGS:', 'GEAR HIGHLIGHTS:']

# Initialize lists to store information
bike_info = []
bags_info = []
gear_info = []

for i in range(len(kit_list)):

    for header in headers:
        # Use regular expression to find the content after each header
        pattern = re.compile(fr'{re.escape(header)}(.*?)(?=\n|$)', re.DOTALL)
        match = pattern.search(kit_list[i])
        
        if match:
            info = match.group(1).strip()
            
            # Append information to the corresponding list
            if header == 'BIKE:':
                bike_info.append(info)
            elif header == 'BAGS:':
                bags_info.append(info)
            elif header == 'GEAR HIGHLIGHTS:':
                gear_info.append(info)

In [12]:
h1_element = soup.find('h1')

# Get the text and remove 'Rigs of ' format snake case

title = h1_element.text.strip('Rigs of ')

print(title)

the 2023 Arizona Trail Race (AZTR)


In [23]:
# Create a dictionary with all the information

race_info = {'Name' : names_riders,
             'Age' : age,
             'Country': country_list,
             'Location' : location,
             'Bike' : bike_info,
             'Bags' : bags_info,
             'Gear' : gear_info,
             'Rance': title,
             'Link': url
             
             }

In [92]:
print("Name:", names_riders)
print("Age:", age)
print("Location:", location)
print('Bike info:', bike_info)
print('Bags info:', bags_info)
print('Gear info:', gear_info)
print('Race')
print('Link:', url)


Name: ['Miron Golfman', 'Matt Annabel', 'Ryan Bartelme', 'Thomas Duehrsen', 'Elliot DuMont', 'Johnny Fitzgerald', 'Zack Freundlich', 'Matan Guttmann', 'Jason Hanson', 'Jennifer Hanson', 'Danny Harper', 'Alexandera Houchin', 'Aaron Johnson', 'Matt Lind', 'Eric Melby', 'John O’Malley', 'Eric Payne', 'Henley Phillips', 'Karin Pocock', 'Steven Powers', 'Johnny Price', 'Ian Quigley', 'Katya Rakhmatulina', 'Dave Roch (Donut King)', 'Kat Roch', 'Elizabeth Sampey', 'Alex Schultz', 'Indiana Schulz', 'Pete Schuster', 'Bill Shand', 'Ryan Sigsbey', 'Steven Spicka', 'Josh Stair', 'Katie Strempke', 'Jeff Taylor', 'Trevor Thornton', 'Kristen Tonsager', 'Josh Tucker', 'Dylan Turner', 'Ethan Washam', 'David Wilson', 'Marissa Wright']
Age: [30, 48, 36, 52, 35, 36, 36, 25, 51, 56, 42, 34, 39, 28, 38, 58, 41, 25, 43, 37, 37, 52, 29, 43, 43, 41, 30, 39, 52, 54, 39, 50, 31, 30, 59, 33, 41, 36, 38, 30, 51, 33]
Location: ['Anchorage Alaska (USA)', 'Carbondale, Colorado (USA)', 'Tucson, Arizona (USA)', 'Thun (

In [14]:
race_info

{'Name': ['Miron Golfman',
  'Matt Annabel',
  'Ryan Bartelme',
  'Thomas Duehrsen',
  'Elliot DuMont',
  'Johnny Fitzgerald',
  'Zack Freundlich',
  'Matan Guttmann',
  'Jason Hanson',
  'Jennifer Hanson',
  'Danny Harper',
  'Alexandera Houchin',
  'Aaron Johnson',
  'Matt Lind',
  'Eric Melby',
  'John O’Malley',
  'Eric Payne',
  'Henley Phillips',
  'Karin Pocock',
  'Steven Powers',
  'Johnny Price',
  'Ian Quigley',
  'Katya Rakhmatulina',
  'Dave Roch (Donut King)',
  'Kat Roch',
  'Elizabeth Sampey',
  'Alex Schultz',
  'Indiana Schulz',
  'Pete Schuster',
  'Bill Shand',
  'Ryan Sigsbey',
  'Steven Spicka',
  'Josh Stair',
  'Katie Strempke',
  'Jeff Taylor',
  'Trevor Thornton',
  'Kristen Tonsager',
  'Josh Tucker',
  'Dylan Turner',
  'Ethan Washam',
  'David Wilson',
  'Marissa Wright'],
 'Age': [30,
  48,
  36,
  52,
  35,
  36,
  36,
  25,
  51,
  56,
  42,
  34,
  39,
  28,
  38,
  58,
  41,
  25,
  43,
  37,
  37,
  52,
  29,
  43,
  43,
  41,
  30,
  39,
  52,
  54,


In [24]:
df_endurance = pd.DataFrame(race_info)
df_endurance.head(10)

,Name,Age,Country,Location,Bike,Bags,Gear,Rance,Link
0,Miron Golfman,30,USA,Anchorage Alaska (USA),"I’m riding a Santa Cruz Blur, with SRAM XO1 dr...","DTF dreams custom handlebar bag, Woolf Tooth b...",KLite Ultra Racing Lighting System w/ Bar swit...,the 2023 Arizona Trail Race (AZTR),https://bikepacking.com/bikes/2023-arizona-tra...
1,Matt Annabel,48,USA,"Carbondale, Colorado (USA)","I’ll be piloting “Badger,” my singlespeed Why ...",A much loved Rogue Panda Canelo bar-harness le...,The dynamo has one job: to juice my Garmin 104...,the 2023 Arizona Trail Race (AZTR),https://bikepacking.com/bikes/2023-arizona-tra...
2,Ryan Bartelme,36,USA,"Tucson, Arizona (USA)",I’ll be riding AZTR300 on a 2021 Yeti SB130 CL...,"If Miles issues an award for bike bag hacks, I...",I brought a small stove for hot coffee and oat...,the 2023 Arizona Trail Race (AZTR),https://bikepacking.com/bikes/2023-arizona-tra...
3,Thomas Duehrsen,52,Switzerland,Thun (Switzerland),2022 ToutTerrain Outback 27.5 steel with Pinio...,Made myself a HF welded framebag from PVC coat...,My Pedalcell broke shortly before the trip and...,the 2023 Arizona Trail Race (AZTR),https://bikepacking.com/bikes/2023-arizona-tra...
4,Elliot DuMont,35,USA,"Tucson, Arizona (USA)","Soma Juice, Shimano 1×12 (30T chainring), home...","Bags by Revelate, Nuclear Sunrise, Diday, and ...",My trusty Brooks B17 saddle and a Mystery Ranc...,the 2023 Arizona Trail Race (AZTR),https://bikepacking.com/bikes/2023-arizona-tra...
5,Johnny Fitzgerald,36,USA,"Helena, Montana (USA)",I’ll be riding a Niner SIR 9 hardtail. This bi...,It’s a mixed bag. Defiant frame bag that is cu...,Bandana for sun cover and cooling. MSR Trail S...,the 2023 Arizona Trail Race (AZTR),https://bikepacking.com/bikes/2023-arizona-tra...
6,Zack Freundlich,36,USA,"McCall, Idaho (USA)","Esker Hayduke Ti, Pike 130, DT Swiss EX Wheels...","Dispersed, Rockgeist, Rogue Panda.","kLite Ultra, Fenix Lights, King Cage, GG Blaze...",the 2023 Arizona Trail Race (AZTR),https://bikepacking.com/bikes/2023-arizona-tra...
7,Matan Guttmann,25,Canada,"North Vancouver, British Columbia (Canada)","I will be riding a 2018 Salsa Timberjack, newl...",Custom Rogue Panda frame bag. Rockgeist Mr. Fu...,I will be wearing a Salomon Active 8 vest and ...,the 2023 Arizona Trail Race (AZTR),https://bikepacking.com/bikes/2023-arizona-tra...
8,Jason Hanson,51,USA,"Florence, Arizona (USA)","Binary SuperBxc, NOX Wheels with Industry Nine...","Broadfork frame bag, Rogue Panda gas tank bag,...",Redshift seat post and Sea to Summit pillow.,the 2023 Arizona Trail Race (AZTR),https://bikepacking.com/bikes/2023-arizona-tra...
9,Jennifer Hanson,56,USA,"Florence, Arizona (USA)","Salsa Spearfish, 1×12 SRAM drivetrain with 26T...","Rogue Panda frame bag, top tube bag and gas ta...",Xerocole Desert hiking boots and Tectonic flat...,the 2023 Arizona Trail Race (AZTR),https://bikepacking.com/bikes/2023-arizona-tra...
